<a href="https://colab.research.google.com/github/ElhamHosseini73/Learn-git/blob/main/recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
import tensorflow_recommenders as tfrs

In [58]:
# Load MovieLens 100k dataset
#The ratings dataset contains information about users, movies, and their interactions (e.g., ratings)
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

In [ ]:
#You can take a look at the first sample from the ratings dataset:
for example in ratings.take(1):
    print(example)


In [ ]:
for movie in movies.take(1):
    print(movie)


In [ ]:
#You need to preprocess the data by extracting relevant fields such as user_id, movie_title, and user_rating.
ratings = ratings.map(lambda x: {
    'user_id': x['user_id'],
    'movie_title': x['movie_title'],
    'user_rating': x['user_rating']
})

for example in ratings.take(1):
    print(example)


In [ ]:
movies = movies.map(lambda x: x['movie_title'])

for movie in movies.take(1):
    print(movie)

In [65]:
# Create vocabularies for users and movies
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(user_ids)

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movie_titles)


In [66]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])


In [67]:
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])


In [68]:
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(movie_model)
    )
)


ValueError: Cannot convert '('c', 'o', 'u', 'n', 't', 'e', 'r')' to a shape. Found invalid entry 'c' of type '<class 'str'>'. 